In [1]:
import os
os.chdir('..')

from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "mean"
fs_method, fs_ratio = "cancelout", 50/142
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from utilities import GNNClassifier

model_fn = functools.partial(
    GNNClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GNNClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9170 | AUPR: 0.6566 | Acc: 95.41%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9222 | AUPR: 0.6729 | Acc: 95.46%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9275 | AUPR: 0.6863 | Acc: 95.50%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9168 | AUPR: 0.6589 | Acc: 95.33%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9233 | AUPR: 0.6760 | Acc: 95.45%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9268 | AUPR: 0.6828 | Acc: 95.49%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9275 | AUPR: 0.6863 | Acc: 95.50%
Internal test: AUC: 0.9313 | AUPR: 0.7069 | Acc: 95.26%
External test: AUC: 0.9207 | AUPR: 0.6686 | Acc: 95.22%


In [3]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9173 | AUPR: 0.6568 | Acc: 95.43%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9225 | AUPR: 0.6712 | Acc: 95.48%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9277 | AUPR: 0.6848 | Acc: 95.50%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9172 | AUPR: 0.6579 | Acc: 95.44%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9215 | AUPR: 0.6710 | Acc: 95.46%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9277 | AUPR: 0.6864 | Acc: 95.61%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9277 | AUPR: 0.6864 | Acc: 95.61%
Internal test: AUC: 0.9324 | AUPR: 0.7082 | Acc: 95.04%
External test: AUC: 0.9217 | AUPR: 0.6741 | Acc: 95.33%


In [4]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9205 | AUPR: 0.6596 | Acc: 95.39%
model(C=10.0)
Valid:         AUC: 0.9204 | AUPR: 0.6587 | Acc: 95.41%
model(C=100.0)
Valid:         AUC: 0.9203 | AUPR: 0.6584 | Acc: 95.40%
best model(C=1.0)
Valid:         AUC: 0.9205 | AUPR: 0.6596 | Acc: 95.39%
Internal test: AUC: 0.9273 | AUPR: 0.6875 | Acc: 95.11%
External test: AUC: 0.9129 | AUPR: 0.6291 | Acc: 94.81%


In [5]:
import warnings
from pytorch_tabnet.tab_model import TabNetClassifier
warnings.filterwarnings("ignore", category=UserWarning)

model_fn = functools.partial(
    TabNetClassifier, seed=SEED, 
    n_d=4, n_a=4, momentum=0.05, 
    n_steps=3, gamma=1.75, cat_emb_dim=1,
    n_independent=2, n_shared=2,
    verbose=0)
param_grid_list = [{
    'optimizer_params': [dict(lr=lr) for lr in [5e-2]],
    'gamma': [1.5, 1.75, 2.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)
warnings.resetwarnings()

TabNetClassifier
model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9135 | AUPR: 0.6373 | Acc: 95.15%
model(optimizer_params={'lr': 0.05}, gamma=1.75)
Valid:         AUC: 0.9115 | AUPR: 0.6461 | Acc: 95.23%
model(optimizer_params={'lr': 0.05}, gamma=2.0)
Valid:         AUC: 0.9045 | AUPR: 0.6260 | Acc: 95.28%
best model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9135 | AUPR: 0.6373 | Acc: 95.15%
Internal test: AUC: 0.9261 | AUPR: 0.6780 | Acc: 95.06%
External test: AUC: 0.9034 | AUPR: 0.6391 | Acc: 94.92%


In [6]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9156 | AUPR: 0.6595 | Acc: 89.65%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9168 | AUPR: 0.6594 | Acc: 89.69%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9174 | AUPR: 0.6625 | Acc: 90.02%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9161 | AUPR: 0.6648 | Acc: 89.51%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9167 | AUPR: 0.6637 | Acc: 89.76%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9175 | AUPR: 0.6661 | Acc: 90.28%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9161 | AUPR: 0.6648 | Acc: 89.51%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9167 | AUPR: 0.6637 | Acc: 89.76%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9175 | AUPR: 0.6661 | Acc: 90.28%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9175 | AUPR: 0.6661 | Ac

In [7]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9195 | AUPR: 0.6358 | Acc: 95.16%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9208 | AUPR: 0.6291 | Acc: 95.07%
model(kernel=linear, C=1)
Valid:         AUC: 0.9203 | AUPR: 0.6263 | Acc: 95.04%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9208 | AUPR: 0.6291 | Acc: 95.07%
Internal test: AUC: 0.9289 | AUPR: 0.6730 | Acc: 94.74%
External test: AUC: 0.9152 | AUPR: 0.6065 | Acc: 94.60%


In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model_fn = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9215 | AUPR: 0.6621 | Acc: 95.34%
model(criterion=squared_error)
Valid:         AUC: 0.9215 | AUPR: 0.6624 | Acc: 95.35%
best model(criterion=squared_error)
Valid:         AUC: 0.9215 | AUPR: 0.6624 | Acc: 95.35%
Internal test: AUC: 0.9304 | AUPR: 0.7008 | Acc: 95.23%
External test: AUC: 0.9192 | AUPR: 0.6623 | Acc: 95.08%


In [9]:
from sklearn.ensemble import AdaBoostClassifier
model_fn = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9163 | AUPR: 0.6362 | Acc: 95.15%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9181 | AUPR: 0.6386 | Acc: 95.16%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9176 | AUPR: 0.6396 | Acc: 95.16%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9181 | AUPR: 0.6386 | Acc: 95.16%
Internal test: AUC: 0.9184 | AUPR: 0.6703 | Acc: 94.77%
External test: AUC: 0.9140 | AUPR: 0.6220 | Acc: 94.74%


In [10]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7673 | AUPR: 0.4464 | Acc: 94.97%
model(n_neighbors=5)
Valid:         AUC: 0.7961 | AUPR: 0.4930 | Acc: 95.07%
model(n_neighbors=7)
Valid:         AUC: 0.8128 | AUPR: 0.5222 | Acc: 95.14%
best model(n_neighbors=7)
Valid:         AUC: 0.8128 | AUPR: 0.5222 | Acc: 95.14%
Internal test: AUC: 0.8156 | AUPR: 0.5616 | Acc: 94.91%
External test: AUC: 0.8135 | AUPR: 0.5402 | Acc: 95.12%


In [11]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8656 | AUPR: 0.5627 | Acc: 88.53%
model(criterion=log_loss)
Valid:         AUC: 0.8640 | AUPR: 0.5735 | Acc: 89.06%
model(criterion=entropy)
Valid:         AUC: 0.8640 | AUPR: 0.5735 | Acc: 89.06%
best model(criterion=gini)
Valid:         AUC: 0.8656 | AUPR: 0.5627 | Acc: 88.53%
Internal test: AUC: 0.8704 | AUPR: 0.6077 | Acc: 90.49%
External test: AUC: 0.8653 | AUPR: 0.5825 | Acc: 89.80%


In [12]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8861 | AUPR: 0.4587 | Acc: 86.74%
best model()
Valid:         AUC: 0.8861 | AUPR: 0.4587 | Acc: 86.74%
Internal test: AUC: 0.8968 | AUPR: 0.5160 | Acc: 86.93%
External test: AUC: 0.8831 | AUPR: 0.4483 | Acc: 85.68%


In [13]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)